# CS542 - Class Challenge - fine-grained classification of plants:

Our class challenge will consists of two tasks addressing an image recognition task where our dataset contains about 1K categories of plants with only about 250,000 images.  There will be two parts to this task:

1. Image classification. Imagine we have cateloged all the plants we care to identify, now we just need to create a classifier for them! Use your skills from the supervised learning sections of this course to try to address this problem.

2. Semi-Supervised/Few-Shot Learning.  Unfortunately, we missed some important plants we want to classify!  We do have some images we think contain the plant, but we have only have a few labels.  Our new goal is to develop an AI model that can learn from just these labeled examples.

Each student must submit a model on both tasks.  Students in the top 3 on each task will get 5% extra credit on this assignment.

This notebook is associated with the second task (semi-supervised).


# Dataset
The dataset is downloaded on scc in the address: "/projectnb2/cs542-bap/classChallenge/data". You can find the python version of this notebook there as well or you could just type "jupyter nbconvert --to script baselineModel_task2.ipynb" and it will output "baselineModel_task2.py". You should be able to run "baselineModel_task2.py" on scc by simply typing "python baselineModel_task2.py"

Please don't try to change or delete the dataset.

# Evaluation:
You will compete with each other over your performance on the dedicated test set. The performance measure is classification accuracy, i.e: if the true class is your top predictions. 

# Baseline:
The following code is a baseline which you can use and improve to come up with your model for this task

# Suggestion
One simple suggestion would be to use a pretrained model on imagenet and finetune it on this data similar to this [link](https://keras.io/api/applications/)
Also you should likely train more than 2 epochs.

## Import TensorFlow and other libraries

In [1]:
total=0
for i in range(1000):
    total += i
    
print(total)

499500


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
print(tf.version.VERSION)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(os.getenv("CUDA_VISIBLE_DEVICES"))
tf.config.set_soft_device_placement(True)
def get_n_cores():
  nslots = os.getenv('NSLOTS')
  if nslots is not None:
    return int(nslots)
  raise ValueError('Environment variable NSLOTS is not defined.')
print("NUM CORES: ", get_n_cores())
tf.config.threading.set_intra_op_parallelism_threads(get_n_cores()-1)
tf.config.threading.set_inter_op_parallelism_threads(1)

2.3.1
Num GPUs Available:  1
3
NUM CORES:  4


# Create a dataset

In [33]:
data_dir = '/projectnb2/cs542-bap/class_challenge/'

train_ds = tf.data.TextLineDataset(os.path.join(data_dir, 'train_held_out_labeled.txt'))
train_unlabeled_ds = tf.data.TextLineDataset(os.path.join(data_dir, 'train_held_out.txt'))
val_ds = tf.data.TextLineDataset(os.path.join(data_dir, 'val_held_out.txt'))
test_ds = tf.data.TextLineDataset(os.path.join(data_dir, 'test_held_out.txt'))

with open(os.path.join(data_dir, 'classes_held_out.txt'), 'r') as f:
  class_names = [c.strip() for c in f.readlines()]

num_classes = len(class_names)
print(num_classes)

20


## Write a short function that converts a file path to an (img, label) pair:

In [34]:
def decode_img(img, crop_size=224):
  img = tf.io.read_file(img)
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # resize the image to the desired size
  return tf.image.resize(img, [crop_size, crop_size])

def get_label(label):
  # find teh matching label
  one_hot = tf.where(tf.equal(label, class_names))
  # Integer encode the label
  return tf.reduce_min(one_hot)

def process_path(file_path):
  # should have two parts
  file_path = tf.strings.split(file_path)
  # second part has the class index
  label = get_label(file_path[1])
  # load the raw data from the file
  img = decode_img(tf.strings.join([data_dir, 'images/', file_path[0], '.jpg']))
  return img, label

def process_path_unlabeled(file_path):
  # load the raw data from the file
  img = decode_img(tf.strings.join([data_dir, 'images/', file_path, '.jpg']))
  #WE are using this method for unlabeled data, so we will manually return -1 as a label for scikit clustering
  return img, tf.cast(-1., tf.int64)

# Finish setting up data

In [35]:
batch_size = 25

# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
train_unlabeled_ds = train_unlabeled_ds.map(process_path_unlabeled, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(process_path, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.map(process_path, num_parallel_calls=AUTOTUNE)

#Concatenate labeled and unlabeled for one training dataset
train_ds = train_ds.concatenate(train_unlabeled_ds)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


## KNN to label unlabled data points

In [36]:
#Split train_ds to X, y array-like for Sci-Kit learn KNN

X = []
y = []
for img, lbl in train_ds:
    X.append(img.numpy().flatten())
    y.append(lbl)


In [37]:
X = np.array(X)
y = np.array(y)

print(X.shape)
print(y.shape)

(4308, 150528)
(4308,)


In [ ]:
#Unfinished Mark

#TRAIN_DS (Labeled) DATA POINTS = 100
#TRAIN__UNLABELED_DS DATA POINTS = 4208
from scipy import stats

from sklearn import datasets
from sklearn.semi_supervised import LabelPropagation

from sklearn.metrics import confusion_matrix, classification_report


n_total_samples = 4308
n_labeled_points = 100


# #############################################################################
# Learn with LabelProp
lp_model = LabelPropagation(kernel='knn', n_neighbors=20, max_iter=20)
lp_model.fit(X, y)

print(y[4208:])

In [ ]:

'''
predicted_labels = lp_model.transduction_[unlabeled_set]
true_labels = y[unlabeled_set]

cm = confusion_matrix(true_labels, predicted_labels, labels=lp_model.classes_)

print("Label Spreading model: %d labeled & %d unlabeled points (%d total)" %
      (n_labeled_points, n_total_samples - n_labeled_points, n_total_samples))

print(classification_report(true_labels, predicted_labels))

print("Confusion matrix")
print(cm)

# #############################################################################
# Calculate uncertainty values for each transduced distribution
pred_entropies = stats.distributions.entropy(lp_model.label_distributions_.T)

# #############################################################################
# Pick the top 10 most uncertain labels
uncertainty_index = np.argsort(pred_entropies)[-10:]

# #############################################################################
# Plot
f = plt.figure(figsize=(7, 5))
for index, image_index in enumerate(uncertainty_index):
    image = images[image_index]

    sub = f.add_subplot(2, 5, index + 1)
    sub.imshow(image, cmap=plt.cm.gray_r)
    plt.xticks([])
    plt.yticks([])
    sub.set_title('predict: %i\ntrue: %i' % (
        lp_model.transduction_[image_index], y[image_index]))

f.suptitle('Learning with small amount of labeled data')
plt.show()

'''

In [ ]:

def augmentFlipUD(img, label):
    return tf.image.flip_up_down(img), label

def augmentFlipLR(img, label):
    return tf.image.flip_left_right(img), label

def augmentRotate(img, label):
    if np.random.random() < .5:
        return tf.image.rot90(img, k=1), label
    else:
        return tf.image.rot90(img, k=3), label
    
#Let's augment and grow our training set 4x

#tdsAugment = train_ds.map(augmentFlipUD, num_parallel_calls=AUTOTUNE)
#tdsAugment = tdsAugment.concatenate(train_ds.map(augmentFlipLR, num_parallel_calls=AUTOTUNE))
#tdsAugment = tdsAugment.concatenate(train_ds.map(augmentRotate, num_parallel_calls=AUTOTUNE))

#We will apply only one augment for training performance
train_ds = train_ds.concatenate(train_ds.map(augmentFlipLR, num_parallel_calls=AUTOTUNE))

## Data loader hyper-parameters for performance!

In [27]:
def configure_for_performance(ds):
  ds = ds.cache()
  ds = ds.shuffle(buffer_size=1000)
  ds = ds.batch(batch_size)
  ds = ds.prefetch(buffer_size=AUTOTUNE)
  return ds

train_ds = configure_for_performance(train_ds)
train_unlabeled_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)
test_ds = configure_for_performance(test_ds)

## A simple CNN model!

In [12]:
resNet50 = keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

for layer in resNet50.layers:
    layer.trainable = False

model = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
  layers.experimental.preprocessing.RandomTranslation(0.2, 0.2),

  layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
  layers.MaxPooling2D((2, 2)),
    
  layers.Conv2D(64, (3, 3), activation='relu'),
  layers.MaxPooling2D((2, 2)),
    
  layers.Dropout(0.5),
    
  layers.Conv2D(128, (3, 3), activation='relu'),
  layers.MaxPooling2D((2, 2)),
    
  layers.Conv2D(256, (3, 3), activation='relu'),
  layers.MaxPooling2D((2, 2)),
    
  layers.Dropout(0.5),
    
  layers.Conv2D(256, (3, 3), activation='relu'),
  layers.MaxPooling2D((2, 2)),
    
  layers.Conv2D(512, (3, 3), activation='relu'),

  layers.Flatten(),
    
  layers.Dense(1024, activation='relu'),
  layers.Dense(num_classes)
])

## The usual loss function

In [13]:
opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(
  optimizer=opt,
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy',tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5)])

## Training

In [14]:
model.fit(train_ds,validation_data=val_ds,epochs=2,shuffle=True)

Epoch 1/2
4/4 [==============================] - 4s 981ms/step - loss: 3.0404 - accuracy: 0.0000e+00 - sparse_top_k_categorical_accuracy: 0.1500 - val_loss: 2.9870 - val_accuracy: 0.0710 - val_sparse_top_k_categorical_accuracy: 0.3894
Epoch 2/2
4/4 [==============================] - 3s 628ms/step - loss: 3.0012 - accuracy: 0.0300 - sparse_top_k_categorical_accuracy: 0.2900 - val_loss: 2.9910 - val_accuracy: 0.0924 - val_sparse_top_k_categorical_accuracy: 0.4076


# Output submission csv for Kaggle


In [23]:
#new_model = keras.models.load_model('baselineAugmented.hdf5')

with open('submission_task2_semisupervised.csv', 'w') as f:
  f.write('id,predicted\n')
  for image_batch, image_names in test_ds:
    predictions = new_model.predict(image_batch)
    for image_name, predictions in zip(image_names.numpy(), new_model.predict(image_batch)):
      inds = np.argmax(predictions)
      line = str(int(image_name)) + ',' + class_names[inds]

InvalidArgumentError: slice index 1 of dimension 0 out of bounds.
	 [[{{node strided_slice}}]]

**Note**

Absolute path is recommended here. For example, use "/projectnb2/cs542-bap/[your directory name]/submission_task2_supervised.csv" to replace "submission_task2_supervised.csv".

Besides, you can request good resources by specify the type of gpus, such as "qsub -l gpus=1 -l gpu_type=P100 [your file name].qsub". This is helpful to avoid potential issues of GPUs, such as out of memory, etc.